<a href="https://colab.research.google.com/github/Hadarlack/Hadarlack/blob/master/Vegan_Vegetarian_GlutenFree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the required libraries

In [0]:
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from IPython.display import clear_output

# Model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC



# Dataset Preparation


In [0]:
del data



In [0]:
del df
del df1
del df2

In [0]:
print ("Read Dataset ... ")

file=files.upload()
data=pd.read_csv("Nutrino Food Label Challenge.csv")



In [0]:
data.head()

In [0]:
df=data # make a copy for convenience 
print("All Data Shape : ", df.shape)


Seperate ingridients : Since Each ingredient is an import feature for the food type, I expand each ingredient into its own variable so that the model can understand whether each ingredient is present or not.

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
list(df['ingredients'].values.tolist()) 


In [0]:
df['ingredients'].head()

In [0]:
 vect = CountVectorizer(tokenizer=lambda x: [i.strip() for i in x.split('//')], lowercase=False) # Convert a collection of text documents to a matrix of token counts


In [0]:
dummies = vect.fit_transform(df['ingredients']) 


In [0]:
# nltk.download()
# nltk.download("stopwords")

In [0]:
# import nltk
# from nltk.corpus import stopwords
 
# set(stopwords.words('english'))
# stopwords = nltk.corpus.stopwords.words('english')
# stopwords.append('regular')
# stopwords.append('cup')
# stopwords.append('chopped')
# stopwords.append('whole')
# stopwords.append('pack')
# stopwords.append('finely')
# stopwords.append('thinly')

# from nltk.corpus import stopwords 
# from nltk.tokenize import word_tokenize 

In [0]:
# word_tokens = df['ingredients'].apply(word_tokenize) 
  
# filtered_sentence = [w for w in word_tokens if not w in stopwords] 
  
# filtered_sentence = [] 
  
# for w in word_tokens: 
#     if w not in stopwords: 
#         filtered_sentence.append(w) 
  
# print(word_tokens) 
# print(filtered_sentence) 

In [0]:
 print((pd.DataFrame(dummies.toarray(),columns = vect.get_feature_names())))

In [0]:
vect.get_feature_names()

In [0]:
df1 = pd.DataFrame(dummies.todense(),columns=vect.get_feature_names())


In [0]:
# list(df1.values.tolist()) 


In [0]:
df1.head()

In [0]:
df2=pd.concat([df,df1],axis=1)


In [0]:
df2.head()

In [0]:
print("All Data Shape: ", df2.shape)


In [0]:
print("Vocab Length: ", len(vect.get_feature_names()))


In [0]:
print("Number of Predictors: ", df.shape[0])


Prepare for Modeling 

In [0]:
df2['GlutenFree'] = pd.get_dummies(df2['GlutenFree'],drop_first=False)
df2['Vegetarian'] = pd.get_dummies(df2['Vegetarian'],drop_first=False)
df2['Vegan'] = pd.get_dummies(df2['Vegan'],drop_first=False)
df2.head()

In [0]:
# from sklearn.preprocessing import OneHotEncoder

# drop_enc = OneHotEncoder(drop='first').fit(df2['foodType'])

In [0]:
df2=df2.drop('ingredients',axis=1) # already processed
df2=df2.drop('name',axis=1) # not helpfull
df2.head()

In [0]:
xx=df2['foodType']
xx.nunique() # 

In [0]:
#The foodType column of the dataset has many categories and we need to reduce the number of categories for a better modelling. The foodType column has the following categories:

# df2.groupby('foodType').apply(lambda x: x.nunique())#.sort_values('name')

# I dont group foodTypes that can be at the same category because they dont have the same predicting values (veg, vig, glooten) and it could confuse the algorithm

In [0]:
# make foodType duumy variables - column for each type with 1/0
cat_vars=['foodType']
for var in cat_vars:
  cat_list='foodType'+'_'+var
  cat_list = pd.get_dummies(df2[var], prefix=var)
  df3=df2.join(cat_list)
  df2=df3
  
cat_vars=['foodType']
data_vars=df2.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]



In [0]:
df4=df2[to_keep]
df4.columns.values

In [0]:
df4.head()

---------------

# Predicting Vegan

--------------------------------------

In [0]:
df4['Vegan'].value_counts() # there are almost twice as much y=1 than y=0

In [0]:
sns.countplot(x='Vegan', data=df4)
plt.show() 

In [0]:
count_no_veg = len(df4[df4['Vegan']==0])
count_veg = len(df4[df4['Vegan']==1])
pct_of_no_veg = count_no_veg/(count_no_veg+count_veg)
print("percentage of no Vegans is", pct_of_no_veg*100)
pct_of_veg = count_veg/(count_no_veg+count_veg)
print("percentage of Vegans", pct_of_veg*100)
#Our classes are imbalanced, and the ratio of 1:0 is 64:36. 

With our data created, I’ll up-sample the no-Vegan of the training set using the SMOTE algorithm (Synthetic Minority Over-sampling Technique)

In [0]:
X = df4.loc[:, df4.columns != 'Vegan']
y = df4.loc[:, df4.columns == 'Vegan']

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

os = SMOTE(random_state=0) #upsampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
columns = X_train.columns
os_data_X,os_data_y=os.fit_sample(X_train, y_train) # upsampling only the training data
X_train = pd.DataFrame(data=os_data_X,columns=columns )
y_train= pd.DataFrame(data=os_data_y,columns=['y'])

In [0]:
print("length of oversampled data is ",len(X_train))
print("Number of no Vegans in oversampled data",len(y_train[y_train['y']==0]))
print("Number of Vegans in oversampled data",len(y_train[y_train['y']==1]))
print("Proportion of no Vegans data in oversampled data is ",len(y_train[y_train['y']==0])/len(X_train))
print("Proportion of Vegans data in oversampled data is ",len(y_train[y_train['y']==1])/len(X_train))


**The models are at the bottum of the code**

--------------------------------------

# Predicting Vegetarian

------------------------------------------------

In [0]:
df4['Vegetarian'].value_counts()  # balanced classes

In [0]:
sns.countplot(x='Vegetarian', data=df4)
plt.show() 

In [0]:
count_no_veg = len(df4[df4['Vegetarian']==0])
count_veg = len(df4[df4['Vegetarian']==1])
pct_of_no_veg = count_no_veg/(count_no_veg+count_veg)
print("percentage of no Vegetarian is", pct_of_no_veg*100)
pct_of_veg = count_veg/(count_no_veg+count_veg)
print("percentage of Vegetarian", pct_of_veg*100)
#Our classes are balanced.


In [0]:
X = df4.loc[:, df4.columns != 'Vegetarian']
y = df4.loc[:, df4.columns == 'Vegetarian']


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

--------------------------------------

# Predicting GlutenFree

------------------------------------------------

In [0]:
df4['GlutenFree'].value_counts() # there are almost twice as much y=0 than y=1

In [0]:
sns.countplot(x='GlutenFree', data=df4)
plt.show() 

In [0]:
count_no_veg = len(df4[df4['GlutenFree']==0])
count_veg = len(df4[df4['GlutenFree']==1])
pct_of_no_veg = count_no_veg/(count_no_veg+count_veg)
print("percentage of no GlutenFree is", pct_of_no_veg*100)
pct_of_veg = count_veg/(count_no_veg+count_veg)
print("percentage of GlutenFree", pct_of_veg*100)


In [0]:
X = df4.loc[:, df4.columns != 'GlutenFree']
y = df4.loc[:, df4.columns == 'GlutenFree']

from imblearn.over_sampling import SMOTE

os = SMOTE(random_state=0) #upsampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
columns = X_train.columns
os_data_X,os_data_y=os.fit_sample(X_train, y_train) # upsampling only the training data
X_train = pd.DataFrame(data=os_data_X,columns=columns )
y_train= pd.DataFrame(data=os_data_y,columns=['y'])

In [0]:
print("length of oversampled data is ",len(X_train))
print("Number of no GlutenFree in oversampled data",len(y_train[y_train['y']==0]))
print("Number of GlutenFree in oversampled data",len(y_train[y_train['y']==1]))
print("Proportion of no GlutenFree data in oversampled data is ",len(y_train[y_train['y']==0])/len(X_train))
print("Proportion of GlutenFree data in oversampled data is ",len(y_train[y_train['y']==1])/len(X_train))


--------------------------------------------------------

# Logistic Regression & Evaluation

---------------------------------------------

In [0]:
logreg = LogisticRegression()

logreg.fit(X_train, y_train)

In [0]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

In [0]:
    from sklearn import metrics

print ("Logistic regression Train Accuracy :: ", metrics.accuracy_score(y_train, logreg.predict(X_train))) #OverFitting
print ("logistic regression Test Accuracy :: ", metrics.accuracy_score(y_test, logreg.predict(X_test)))    
    

In [0]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

In [0]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [0]:
# submission = logreg.predict(X_test)
score = cross_validate(logreg, X_test, y_test, return_train_score=False)
score["test_score"].mean()


In [0]:
# # The receiver operating characteristic (ROC) curve 

# from sklearn.metrics import roc_auc_score
# from sklearn.metrics import roc_curve

# logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
# fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
# plt.figure()
# plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
# plt.plot([0, 1], [0, 1],'r--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic')
# plt.legend(loc="lower right")
# plt.savefig('Log_ROC')
# plt.show()

------------------------------------------------

Evaluation

--------------------------------------------------------------------

In [0]:
# from sklearn.metrics import classification_report
# print(classification_report(y_test,predictions))

In [0]:
# print(y_test)

In [0]:
# # Evaluate using Cross Validation

# from sklearn import model_selection

# seed=7
# num_instances = len(X_train)

# kfold = model_selection.KFold(n_splits=10, random_state=seed)
# # model = LogisticRegression()
# results = model_selection.cross_val_score(logreg, X_train, y_train, cv=kfold)
# print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

In [0]:
# # Evaluate using Leave One Out Cross Validation


# num_folds = 5
# num_instances = len(X_train)
# loocv = model_selection.LeaveOneOut()
# # model = LogisticRegression()
# results = model_selection.cross_val_score(logmodel, X_train, y_train, cv=loocv)
# print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

In [0]:
# score = cross_validate(logreg, X_train, y_train, return_train_score=False)
# score["test_score"].mean()